In [4]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('datasets/train.csv', delimiter=';')
dataset_gpt1 = pd.read_csv('datasets/gpt1.csv', delimiter=';')
dataset_gpt2 = pd.read_csv('datasets/gpt2.csv', delimiter=';')
dataset_gpt3 = pd.read_csv('datasets/gpt3.csv', delimiter=';')
dataset_gpt4 = pd.read_csv('datasets/gpt4.csv', delimiter=';')
dataset_gpt5 = pd.read_csv('datasets/gpt5.csv', delimiter=';')
dataset_gpt6 = pd.read_csv('datasets/gpt6.csv', delimiter=';')

datasets = [dataset_gpt1, dataset_gpt2, dataset_gpt3, dataset_gpt4, dataset_gpt5, dataset_gpt6]
test_count = 6000

In [11]:
data = dataset.drop_duplicates()
data = data.dropna()
data = data[data['Текст инцидента'] != '\'Сообщение без текста']
for msg in data['Текст инцидента']:
    if len(msg.split(' ')) <= 2:    
        data = data[data['Текст инцидента'] != msg]

data_test = data.iloc[:test_count, :]
data_train = data.iloc[test_count:, :]

for d in datasets:
    d = d.drop_duplicates()
    d = d.dropna()
    d = d[d['Текст инцидента'] != '\'Сообщение без текста']
    for msg in d['Текст инцидента']:
        if len(msg.split(' ')) <= 2:    
            d = d[d['Текст инцидента'] != msg]
    
    data_train = data_train._append(d)


def get_group_code():
    groups = dataset['Группа тем'].unique()
    dict = {groups[i]:i for i in range(0, len(groups))}
    return dict

group_codes = get_group_code()

def get_themes_code():
    themes = dataset['Тема'].unique()
    dict = {themes[i]:i for i in range(0, len(themes))}
    return dict

themes_codes = get_themes_code()

group_test = data_test['Группа тем']
group_test = np.array(group_test.replace(group_codes))
theme_test = data_test['Тема']
theme_test = np.array(theme_test.replace(themes_codes))

group_train = data_train['Группа тем']
group_train = np.array(group_train.replace(group_codes))
theme_train = data_train['Тема']
theme_train = np.array(theme_train.replace(themes_codes))



data_train.shape

(44518, 4)

In [17]:
#for get new embeddings from another dataset
'''
import torch
from transformers import AutoTokenizer, AutoModel

import pymorphy3
from razdel import tokenize
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

morph = pymorphy3.MorphAnalyzer()

stopTags = ['PNCT', 'NUMB', 'UNKN', 'LATN', 'ROMN']

def preprocess_text(text):
    tokens = tokenize(text.lower())
    tokens = [morph.parse(token.text)[0].normal_form for token in tokens 
              if morph.parse(token.text)[0].normal_form not in russian_stopwords and 
              not any(tag in morph.parse(token.text)[0].tag for tag in stopTags)]
    text = " ".join(tokens) 
    return text

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

texts = []

list = []

for s in texts:
    s = preprocess_text(s)
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=256, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    emb = model_output.pooler_output
    list.append((emb)[0].numpy())

x_embeddings = np.asarray(list)

np.save(f'embeddings', x_embeddings)
'''

' 22554\nimport torch\nfrom transformers import AutoTokenizer, AutoModel\n\ntokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")\nmodel = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")\n\nlist = []\n\nfor s in x:\n    s = preprocess_text(s)\n    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=256, return_tensors=\'pt\')\n    with torch.no_grad():\n        model_output = model(**encoded_input)\n    emb = model_output.pooler_output\n    list.append((emb)[0].numpy())\n\nx_embeddings = np.asarray(list)\n\n#сохранить embeddings\nnp.save(f\'embeddings_original\', x_embeddings)\n'

In [9]:
#for get embeddings from datasets
embeddings = np.load('embeddings/embeddings_original.npy')
embeddings_test = embeddings[:test_count]
embeddings_gpt1 = np.load('embeddings/embeddings_gpt1.npy')
embeddings_gpt2 = np.load('embeddings/embeddings_gpt2.npy')
embeddings_gpt3 = np.load('embeddings/embeddings_gpt3.npy')
embeddings_gpt4 = np.load('embeddings/embeddings_gpt4.npy')
embeddings_gpt5 = np.load('embeddings/embeddings_gpt5.npy')
embeddings_gpt6 = np.load('embeddings/embeddings_gpt6.npy')

embeddings_train = np.concatenate([embeddings[test_count:], embeddings_gpt1, 
                                     embeddings_gpt2, embeddings_gpt3, embeddings_gpt4, 
                                     embeddings_gpt5, embeddings_gpt6])
embeddings_train.shape

(44518, 768)

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score

x_group_train, x_group_test, y_group_train, y_group_test = embeddings_train, embeddings_test, group_train, group_test

x_theme_train, x_theme_test, y_theme_train, y_theme_test = embeddings_train, embeddings_test, theme_train, theme_test

x_theme_train = np.hstack((x_theme_train, y_group_train.reshape(-1, 1)))
x_theme_test = np.hstack((x_theme_test, y_group_test.reshape(-1, 1)))

In [20]:
group_model = SVC(C = 3, random_state=10)
group_model.fit(x_group_train, y_group_train)

y_group_pred = group_model.predict(x_group_test)

f1_group = f1_score(y_group_test, y_group_pred, average='weighted')
print('F1_GROUP: ', round(f1_group, 3))

F1_GROUP:  0.833


In [21]:
theme_model = SVC(C = 4, random_state=10)
theme_model.fit(x_theme_train, y_theme_train)

y_theme_pred = theme_model.predict(x_theme_test)

f1_theme = f1_group * f1_score(y_theme_test, y_theme_pred, average='weighted')
print('F1_THEME: ', round(f1_theme, 3))

In [ ]:
from joblib import dump, load
#dump(group_model, 'group_model.joblib')
#dump(theme_model, 'theme_model.joblib')


['theme_model.joblib']

### F1_GROUP:  0.835
### F1_THEME:  0.744 0.621
